In [1]:
import os
import json
import pandas as pd

In [2]:
folder_path = '불안장애_1-10'

In [3]:
output_folder = './불안장애전처리'

In [4]:
import re

cols = ['paragraph_text', 'paragraph_speaker', 'anxiety_mood','derealization','perceived_loss_of_control','anxiety_control','concentration','avoidance','physical_symptoms','irritability']
symptom_cols = ['anxiety_mood','derealization','perceived_loss_of_control','anxiety_control','concentration','avoidance','physical_symptoms','irritability']

def is_survey_or_rating(text):
    survey_keywords = ['네', '아니요', '모름', '없음', '예', '아니오', '있음', '잘 모르겠음']
    rating_pattern = r'^[0-9]+점\\.?$'
    text_clean = str(text).strip()
    if re.match(rating_pattern, text_clean):
        return True
    if text_clean in survey_keywords:
        return True
    return False

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            paragraphs = data['paragraph']
            df = pd.DataFrame(paragraphs)
            df_selected = df[cols]

            # 설문/평가 텍스트 행 제거
            df_selected = df_selected[~df_selected['paragraph_text'].apply(is_survey_or_rating)]

            # 지정 증상 컬럼 중 하나라도 0이 아닌 행만 필터링
            df_filtered = df_selected[(df_selected[symptom_cols] != 0).any(axis=1)]

            output_path = os.path.join(output_folder, filename.replace('.json', '.csv'))
            df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'{filename}: 완료, {len(df_filtered)}개 행')

        except Exception as e:
            print(f'{filename}: 에러 발생 - {str(e)}')
            continue

label_anxiety_10_check_X052.json: 완료, 8개 행
label_anxiety_1_check_X002.json: 완료, 24개 행
label_anxiety_1_check_X004.json: 완료, 20개 행
label_anxiety_1_check_X005.json: 완료, 8개 행
label_anxiety_1_check_X006.json: 완료, 29개 행
label_anxiety_1_check_X007.json: 완료, 25개 행
label_anxiety_1_check_X008.json: 완료, 20개 행
label_anxiety_1_check_X009.json: 완료, 33개 행
label_anxiety_1_check_X012.json: 완료, 32개 행
label_anxiety_1_check_X013.json: 완료, 17개 행
label_anxiety_1_check_X015.json: 완료, 39개 행
label_anxiety_1_check_X017.json: 완료, 28개 행
label_anxiety_1_check_X018.json: 완료, 38개 행
label_anxiety_1_check_X019.json: 완료, 27개 행
label_anxiety_1_check_X022.json: 완료, 10개 행
label_anxiety_1_check_X024.json: 완료, 28개 행
label_anxiety_1_check_X026.json: 완료, 29개 행
label_anxiety_1_check_X028.json: 완료, 26개 행
label_anxiety_1_check_X029.json: 완료, 31개 행
label_anxiety_1_check_X032.json: 완료, 19개 행
label_anxiety_1_check_X033.json: 완료, 13개 행
label_anxiety_1_check_X034.json: 완료, 3개 행
label_anxiety_1_check_X035.json: 완료, 41개 행
label_anxiety

In [ ]:
!pip install -qU konlpy
!pip install -qU sentence_transformers
# Python 3.11.9 버전으로 구동할 것

In [3]:
import os
import re
import pandas as pd
from konlpy.tag import Okt
from sentence_transformers import SentenceTransformer

# 1. 텍스트 정규화 함수
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# 2. 한국어 토큰화
okt = Okt()
def tokenize(text):
    return okt.morphs(text)

# 3. 벡터 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

# 처리 대상 폴더
folder_path = '불안장애전처리'  # CSV 파일들이 위치한 폴더 경로

# 결과 저장용 딕셔너리(파일명 -> 문장별 임베딩)
all_embeddings = {}

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        # 'paragraph_text' 컬럼 기준 처리
        normalized_texts = df['paragraph_text'].astype(str).apply(normalize_text)
        tokenized_texts = normalized_texts.apply(tokenize).apply(lambda tokens: ' '.join(tokens))

        # 임베딩 생성
        embeddings = embedding_model.encode(tokenized_texts.tolist())

        # 결과 저장
        all_embeddings[file] = embeddings

        print(f'{file} 처리 완료, 문장 수: {len(normalized_texts)}')

for file in os.listdir(folder_path):
    # 확장자가 .csv인 정상 파일만 처리, 임시파일명(~$...) 제외
    if file.endswith('.csv') and not file.startswith('~$'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        # 이후 처리 계속

# all_embeddings는 각 파일별로 문장 임베딩 배열 저장


label_anxiety_10_check_X052.csv 처리 완료, 문장 수: 8
label_anxiety_1_check_X002.csv 처리 완료, 문장 수: 24
label_anxiety_1_check_X004.csv 처리 완료, 문장 수: 20
label_anxiety_1_check_X005.csv 처리 완료, 문장 수: 8
label_anxiety_1_check_X006.csv 처리 완료, 문장 수: 29
label_anxiety_1_check_X007.csv 처리 완료, 문장 수: 25
label_anxiety_1_check_X008.csv 처리 완료, 문장 수: 20
label_anxiety_1_check_X009.csv 처리 완료, 문장 수: 33
label_anxiety_1_check_X012.csv 처리 완료, 문장 수: 32
label_anxiety_1_check_X013.csv 처리 완료, 문장 수: 17
label_anxiety_1_check_X015.csv 처리 완료, 문장 수: 39
label_anxiety_1_check_X016.csv 처리 완료, 문장 수: 41
label_anxiety_1_check_X017.csv 처리 완료, 문장 수: 28
label_anxiety_1_check_X018.csv 처리 완료, 문장 수: 38
label_anxiety_1_check_X019.csv 처리 완료, 문장 수: 27
label_anxiety_1_check_X022.csv 처리 완료, 문장 수: 10
label_anxiety_1_check_X024.csv 처리 완료, 문장 수: 28
label_anxiety_1_check_X026.csv 처리 완료, 문장 수: 29
label_anxiety_1_check_X027.csv 처리 완료, 문장 수: 5
label_anxiety_1_check_X028.csv 처리 완료, 문장 수: 26
label_anxiety_1_check_X029.csv 처리 완료, 문장 수: 31
label_anxiety_1

In [4]:
import pickle

with open('anxdata_all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)
    
with open('anxdata_all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)